In [1]:
import cv2
import numpy as np
import glob
import os
import cv2
import pandas as pd

import matplotlib.pyplot as plt

from comb_registration import keypoints_match_image, get_upper_keypoints
from comb_registration import show_keypoints_on_image
from comb_registration import get_warp_matrix
from comb_registration import load_reference_mask
from comb_registration import load_reference_image
from comb_registration import load_reference_keypoints
from comb_registration import load_keypoints
from comb_registration import mirror_keypoints_horizontal
    
import sys
sys.path.append('../functions')

from comb_loading import get_organized_colony_names
from comb_loading import load_colony_comb  
from mask_processing_test import visualize_get_distance_to_class

In [2]:
base_folder = "/home/golden/kenya_drones/comb-detection/nest_photos"

reference_image = load_reference_image(base_folder, resize=True)
reference_mask = load_reference_mask(base_folder, resize=False)
reference_keypoints = load_reference_keypoints(base_folder, resize=False, 
                                               return_upper=True
                                              )
if False:
    show_keypoints_on_image(reference_image, reference_keypoints)
    show_keypoints_on_image(reference_mask, reference_keypoints)

In [3]:
meta_file = "/home/golden/Dropbox/bee-comb/img_to_text_df_TOEDIT.csv"
beeframe_meta = pd.read_csv(meta_file)
colony_names = get_organized_colony_names(beeframe_meta)

In [51]:
masks_folder_name = "masks"
new_masks_folder_name = "ab_aligned_masks"

In [57]:
visualize = False

side_b_meta = beeframe_meta.loc[beeframe_meta['side'] == 'b']
for index, row in side_b_meta.iterrows():
    if index % 100 == 0:
        print(index)
    colony_name = row['colony']
    date = row['date']
    beeframe = row['beeframe']
    
    date_folder = os.path.join(base_folder, colony_name, str(date))
    b_frame_name = os.path.splitext(row['filename'])[0]
    b_maskfile = os.path.join(date_folder, masks_folder_name, 
                              f"{b_frame_name}.png"
                             )
    b_mask = cv2.imread(b_maskfile, cv2.IMREAD_GRAYSCALE)
    if b_mask is not None:
        b_mask = b_mask[:, ::-1]
        b_side_keypoints_file = os.path.join(base_folder, colony_name, 
                                            str(date), "keypoints", 
                                            f"{b_frame_name}.csv"
                                           )
        b_side_keypoints = load_keypoints(b_side_keypoints_file, 
                                          return_upper=True
                                         )
        b_side_keypoints = mirror_keypoints_horizontal(b_side_keypoints, 
                                                       b_mask.shape[1]
                                                      )

        transform = get_warp_matrix(b_side_keypoints, reference_keypoints)
        b_mask_warped = cv2.warpAffine(b_mask, M=transform, 
                                       dsize=(reference_mask.shape[1], 
                                              reference_mask.shape[0]
                                             )
                                      )
        b_maskfile_aligned = b_maskfile.replace(masks_folder_name, 
                                                new_masks_folder_name
                                               )
        os.makedirs(os.path.dirname(b_maskfile_aligned), exist_ok=True)
        cv2.imwrite(b_maskfile_aligned, b_mask_warped)
    a_ind = ((beeframe_meta['date']==date)
             & (beeframe_meta['colony']==colony_name)
             & (beeframe_meta['beeframe']==beeframe)
             & (beeframe_meta['side']=='a')
            )
    if np.sum(a_ind) == 1:
        a_frame_filename = beeframe_meta.loc[a_ind, 'filename'].item()
        a_frame_name = os.path.splitext(a_frame_filename)[0]
        a_maskfile = os.path.join(date_folder, 
                                  f"warped_{masks_folder_name}", 
                                  f"{a_frame_name}.png"
                                 )
        a_mask = cv2.imread(a_maskfile, cv2.IMREAD_GRAYSCALE)
        if a_mask is not None:
            a_mask = cv2.resize(a_mask, 
                                (reference_mask.shape[1], reference_mask.shape[0]),
                                interpolation=cv2.INTER_LINEAR
                               )
            a_maskfile_aligned = a_maskfile.replace(f"warped_{masks_folder_name}", 
                                                    new_masks_folder_name
                                                   )
            os.makedirs(os.path.dirname(a_maskfile_aligned), exist_ok=True)
            cv2.imwrite(a_maskfile_aligned, a_mask)
            if visualize:
                fig, axs = plt.subplots(1, 2)
                axs[0].imshow(a_mask)
                axs[0].imshow(b_mask_warped[:,::], alpha=.5)
                axs[1].imshow(a_mask - b_mask_warped)